## Concurrency, parallelism and python

### Process

<b>Процесс</b> — запущенная программа.
* У каждого процесса есть изолированное от других
процессов состояние:
    * виртуальное адресное пространство,
    * указатель на исполняемую инструкцию,
    * стек вызовов,
    * системные ресурсы, например, открытые файловые
        дескрипторы.
* Процессы удобны для одновременного выполнения
нескольких задач.
* Альтернативный способ: делегировать каждую задачу на
выполнение потоку

### Thread

Поток похож на процесс тем, что его исполнение
происходит независимо от других потоков (и процессов).
* В отличие от процесса поток исполняется внутри процесса
и разделяет с ним адресное пространство и системные
ресурсы.
* Потоки удобны для одновременного выполнения
нескольких задач, которым требуется доступ к
разделяемому состоянию.
* Совместным выполнением нескольких процессов и
потоков управляет операционная система, поочерёдно
разрешая каждому процессу или потоку использовать
сколько-то циклов процессора.

In [5]:
import time
import requests

def fetch_response(url: str) -> requests.Response:
    return requests.get(url)

def print_status_codes(responses: list):
    for r in responses:
        print(r.status_code, end=" ")

In [6]:
urls = [
    "https://docs.python.org/3/",
    "https://google.com",
    "https://example.com",
    "https://python.org",
    "https://www.youtube.com/",
    "https://github.com/",
]

### Simple (synchronously)

In [8]:
%%time
responses = [fetch_response(url) for url in urls]
print_status_codes(responses) 

200 200 200 200 200 200 CPU times: user 155 ms, sys: 19.2 ms, total: 174 ms
Wall time: 2.52 s


### multithreading

In [9]:
from concurrent.futures import ThreadPoolExecutor

In [12]:
%%time
# max_workers is changed to min(32, os.cpu_count() + 4) python3.8+
with ThreadPoolExecutor(max_workers=6) as pool:
    responses = pool.map(fetch_response, urls)
    print_status_codes(responses)

200 200 200 200 200 200 CPU times: user 230 ms, sys: 26.8 ms, total: 257 ms
Wall time: 769 ms


### multiprocessing

In [3]:
from concurrent.futures import ProcessPoolExecutor

In [1]:
%%time
# If max_workers is None or not given, it will default to the number of processors (cores) on the machine
with ProcessPoolExecutor(max_workers=len(urls)) as pool:
    responses = pool.map(fetch_response, urls)
    print_status_codes(responses)

NameError: name 'ProcessPoolExecutor' is not defined

### Another example

In [5]:
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419,
]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

### Simple (synchronously)

In [6]:
%%time
for number, _is_prime in zip(PRIMES, map(is_prime, PRIMES)):
    print(f'{number} is prime: {_is_prime}')

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
CPU times: user 2.77 s, sys: 22.6 ms, total: 2.79 s
Wall time: 2.82 s


### multithreading

In [9]:
%%time
with ThreadPoolExecutor() as pool:
    for number, _is_prime in zip(PRIMES, pool.map(is_prime, PRIMES)):
            print(f'{number} is prime: {_is_prime}')

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
CPU times: user 2.75 s, sys: 46 ms, total: 2.8 s
Wall time: 2.8 s


### multiprocessing

In [4]:
%%time
with ProcessPoolExecutor() as pool:
    for number, _is_prime in zip(PRIMES, pool.map(is_prime, PRIMES)):
            print(f'{number} is prime: {_is_prime}')

NameError: name 'ProcessPoolExecutor' is not defined

### Why is there such a big difference?

### parallelism and concurrency

![](https://s3.tproger.ru/uploads/2016/12/concepts3mini.png)

### Why is there such a big difference?
### Answer is GIL

* <b>GIL</b> (global interpreter lock) — это мьютекс, который гарантирует, что в каждый момент времени только один
поток имеет доступ к состоянию интерпретатора.
* Python C API позволяет отпустить GIL, но это безопасно только при работе с объектами, не зависящими от интерпретатора Python.
* Например, все операции ввода/вывода в CPython отпускают GIL
* переключение контекста каждые 5 миллисекунд (вытесняющая, принудительная многозадачность)

In [ ]:
...
Py_BEGIN_ALLOW_THREADS
err = close(fd);
if (err < 0)
    save_errno = errno;
Py_END_ALLOW_THREADS
...

### So GIL is bad?

### Nope! Depends on task

### Have a snack?
![](https://upload.wikimedia.org/wikipedia/commons/3/36/McDonald%27s_Golden_Arches.svg)

### Lets order a coffee

![](https://i.imgur.com/XsSm55k.png)

### Look good, now lets order a Big Mac


![](https://cdn-images-1.medium.com/max/720/0*APVcPuyDaIKSDZPz.png)


### Where is the relation?

![](https://i.imgur.com/3zAfnP7.png)

### I/O bound tasks vs CPU bound tasks
![](https://i.imgur.com/8F3Roo8.png)

### GIL doesn't save you from logical errors

In [6]:
for i in range(1000):
    counter = 0

    def increment_counter(fake_value):
        global counter
        for _ in range(100):
            counter = counter + 1


    fake_data = [x for x in range(5000)]
    with ThreadPoolExecutor(max_workers=5000) as executor:
        executor.map(increment_counter, fake_data)
    if counter != 500000:
        print(i," -- ", counter)

402  --  498200


In [ ]:
from threading import Lock, RLock
lock = RLock()

for i in range(1000):
    counter = 0

    def increment_counter(fake_value):
        global counter
        for _ in range(100):
            with lock:
                counter += 1


    fake_data = [x for x in range(5000)]
    with ThreadPoolExecutor(max_workers=5000) as executor:
        executor.map(increment_counter, fake_data)
    if counter != 500000:
        print(counter)

### + multithreading is hardly scalable

![](https://files.realpython.com/media/Screen_Shot_2018-10-17_at_3.18.44_PM.c02792872031.jpg)

### Materials
#### Documentation:
* [Concurrent Execution in python](https://docs.python.org/3/library/concurrency.html)
* [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html)
* [multiprocessing ](https://docs.python.org/3/library/multiprocessing.html)
* [asyncio](https://docs.python.org/3/library/asyncio.html)
* [Celery](https://docs.celeryproject.org/en/stable/getting-started/introduction.html)
####  Videos to watch
* [McDonalds и Python](https://www.youtube.com/watch?v=b3iXdDmXm7s&t=167s&ab_channel=%D0%94%D0%B8%D0%B4%D0%B6%D0%B8%D1%82%D0%B0%D0%BB%D0%B8%D0%B7%D0%B8%D1%80%D1%83%D0%B9%21)
* [Многопроцессность, многопоточность, асинхронность в Python](https://www.youtube.com/watch?v=JIp14T9bvvc&ab_channel=%D0%94%D0%B8%D0%B4%D0%B6%D0%B8%D1%82%D0%B0%D0%BB%D0%B8%D0%B7%D0%B8%D1%80%D1%83%D0%B9%21)
* [Многопоточность и GIL](https://www.youtube.com/watch?v=nR8WhdcRJwM&ab_channel=ComputerScienceCenter)
* [GIL в Python: зачем он нужен и как с этим жить](https://www.youtube.com/watch?v=AWX4JnAnjBE&ab_channel=MoscowPython)
* [Андрей Светлов: "Подводные камни asyncio"](https://www.youtube.com/watch?v=GLN_xo4Awcc&ab_channel=PiterPy)
#### Real python ultimate guides
* [Speed Up Your Python Program With Concurrency](https://realpython.com/python-concurrency/)
* [Async IO in Python: A Complete Walkthrough](https://realpython.com/async-io-python/)
#### Others
* [Асинхронное программирование в Python](https://tproger.ru/translations/asynchronous-programming-in-python/)
* [Зачем, когда и как использовать multithreading и multiprocessing в Python](https://habr.com/ru/company/otus/blog/501056/)
* [Asynchronous programming. Python3.5+](https://luminousmen.com/post/asynchronous-programming-python3.5)
* [How does asyncio work?](https://stackoverflow.com/questions/49005651/how-does-asyncio-actually-work/51116910#51116910)
* [Concurrent computing (by Andrey Solomatin)](https://github.com/Cjkjvfnby/presentation-async/blob/master/src/main/asciidoc/presentation.adoc)

### Homework
Ваша задача спарсить информацию о компаниях, находящихся в индексе S&P 500 с данного сайта: <br>
https://markets.businessinsider.com/index/components/s&p_500

Для каждой компании собрать следующую информацию:
* Текущая стоимость в рублях (конвертацию производить по текущему курсу, взятому с сайта [центробанка РФ](http://www.cbr.ru/development/sxml/))
* Код компании (справа от названия компании на странице компании)
* P/E компании (информация находится справа от графика на странице компании)
* Годовой рост/падение компании в процентах (основная таблица)
* Высчитать какую прибыль принесли бы акции компании (в процентах), если бы они были куплены на уровне 52 Week Low и проданы на уровне 52 Week High (справа от графика на странице компании)

Сохранить итоговую информацию в 4 JSON файла:
1. Топ 10 компаний с самими дорогими акциями в рублях.
2. Топ 10 компаний с самым низким показателем P/E.
3. Топ 10 компаний, которые показали самый высокий рост за последний год
4. Топ 10 комппаний, которые принесли бы наибольшую прибыль, если бы были куплены на самом минимуме и проданы на самом максимуме за последний год.
<br>Пример формата:
```
[
{
    "code": "MMM",
    "name": "3M CO.",
    "price" | "P/E" | "growth" | "potential profit" : value,
},
...
]
```
<br>**P.S. по-максимуму использовать возможности параллелизма и асинхронности.** 
<br>
* bs4
* aiohttp